# Spectral clustering - basic version

### 1. Importing data
#### 1.1 Artificial data
Directories in "Project1" directory:
- https://github.com/gagolews/clustering-benchmarks
- https://github.com/gagolews/clustering-data-v1

In [58]:
# !pip install natsort
# !pip install genieclust

In [59]:
from artificial_data import *
X, labels = importBiggerArtificialData()
# X, labels = importSmallerArtificialData()


In [60]:
labels[0]

array([3, 3, 3, ..., 2, 2, 2])

#### 1.2 Real data
Data obtained from https://snap.stanford.edu/data/gemsec-Facebook.html

In [61]:
from real_data import *
# X = importFacebookTvShowData()
# X = importFacebookArtistData()

### 2. Importing libraries

In [62]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
import numpy as np
import math

### 3. Data pre-processing
#### 3.1 Faithful sampling
Following functionalities are responsible for performing faithful sampling on the originl artifical data set. The aim is to reduce the size of the data by aggregating the points into representative communities

In [63]:
# Function marks a given point as registered
#
# pt - point to be marked as registered
# data - data set
def markPointRegistered(pt, data):
    # selecting corresponding point in data set
    mask = np.apply_along_axis(lambda x: np.array_equal(x[0], pt), 1, data)
    
    # marking point as registered
    temp = data[mask]                                                       
    temp[:, 1] = True
    data[mask] = temp


# Function initializes the data by marking all points as unregistered
#
# data - data set
# return: 2D ndarray of the form: [[<point coord.>], False]
def initializeUnregisteredPoints(X):
    return np.apply_along_axis(lambda x: np.array([x, False], dtype=object), 1, X)


In [64]:
# Function executes faithful sampling algorithm
# (based on: https://bmcbioinformatics.biomedcentral.com/counter/pdf/10.1186/1471-2105-11-403.pdf)
#
# X - data set
# m - maximal number of communities in a single cluster (initially 1000)
# return: list of points aggregated in communities
def faithfulSampling(X, m = 100):

    dim = len(X[0])              # dimension of the data set
    n = len(X)                   # size of the data set
    V = n * dim                  # space volume

    h = (1/2) * (V/m)**(1/dim)   # initializing h
    m1 = m                       # setting initial communities size

    
    # iterate untill size of h will no longer be adjustable
    while((m/2) <= m1 and m1 <= m):

        data = initializeUnregisteredPoints(X)                      # mark all data points as unregistered
        communities = []                                            # initialize communities buckets
        
        # iterate until all data points are registered
        while(np.any(data[:, 1] == False)):

            unregistered = data[data[:, 1] == False][:, 0]          # taking unregistered data
            randomPt = np.random.choice(unregistered, 1)[0]         # selecting random data point

            community = np.empty((0, dim))                             # initializing local community

            for pt in unregistered:
                distance = np.linalg.norm(pt - randomPt)            # computing euclidean distance

                if(distance < h):
                    markPointRegistered(pt, data)                    # marking point as registered
                    community = np.append(community, [pt], axis=0)   # adding point to local community

            communities.append(community)
        
        m1 = len(communities)       # computing new maximal community length
        h = h * (m1/m)**(1/dim)     # adjusting maximal distance

    print(f"Number of communities: {len(communities)}")
    return communities

### 4. Spectral clustering
#### 4.1 Compute similarity graph
The first step considers constructing the similarity graph. Here, two approaches were implemented:
- KNN (K Nearest Neighbours) graph
- graph constructed based on the similarity measure from *faithful sampling*

In [ ]:
# Function constructs the graph representation by connecting the vertices using KNN approach
#
# X - data set
# M - number of neighbours (initially 5)
# return: ndarray adjacency matrix representing the graph
def adjacencyMatrixUsingMnearestNeighbors(X, M = 5):
    
    knn = NearestNeighbors(n_neighbors=M)
    knn.fit(X)
    dist_indx_arr = knn.kneighbors(X, return_distance=True, n_neighbors = M)
    
    A = kneighbors_graph(X, n_neighbors=(M-1), p=2, mode='connectivity', include_self=False)
    return A.toarray(), dist_indx_arr # dist_indx_arr will be useful while connecting the graph

# Function constructs the graph representation by using similarity measures computed on communities obtained after faithful sampling
#
# communities - communities of points
# def adjacencyMatrixUsingSimilarityMeasures(communities):
def adjacencyMatrixUsingSimilarityCommunities(communities):
    # TODO
    return communities


In [ ]:
def connectTheGraph(A, dist_indx_arr=None):
    
    # to be implemented
    
    return A

def graphLaplacian(A):
    D = np.eye(A.shape[0]) * A.sum(axis=0) # diagonal matrix of degrees
    return D  - A # L = D - A
    

def calculateEigenVectorsOfGraphLaplacian(L):
    eigenValues, eigenVectors = np.linalg.eig(L)
    eigenValues, eigenVectors = eigenValues.real, eigenVectors.real
    
    idx = eigenValues.argsort()[::1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    return eigenValues,eigenVectors
    
    
    
def nodeRepresentation(eigenVectors, nodeRepresentationDim, n_of_components=1):
    m = n_of_components # we assume n of connected components = 1
    Z = eigenVectors[:,m:(nodeRepresentationDim + m)] # we omit m first eigenvectors, where m is the number of components of graph from A
    return Z
    
    

Number of communities: 1


In [ ]:
from sklearn.cluster import KMeans

X, labels = importSmallerArtificialData()


def spectralClustering(X, n_of_clusters=3, M=3, nodeRepresentationDim=3):
    
    A, d = adjacencyMatrixUsingMnearestNeighbors(X, M=M)
    A_connected = connectTheGraph(A, d)
    L = graphLaplacian(A_connected)
    w,v = calculateEigenVectorsOfGraphLaplacian(L)
    Z = nodeRepresentation(v,nodeRepresentationDim,n_of_components=1)
    

    print(Z.shape)
    kmeans = KMeans(n_clusters=n_of_clusters, random_state=0, n_init=100).fit(Z)
    return kmeans.labels_





In [ ]:
spectralClustering(X, n_of_clusters=3, nodeRepresentationDim=3, M=10)

(192, 3)


array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1,
       2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0,
       1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1,
       1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [ ]:
from sklearn.metrics import adjusted_rand_score as AR

spectral_labels = spectralClustering(X, n_of_clusters=3, nodeRepresentationDim=3, M=10)

AR(spectral_labels, labels[0])

(192, 3)


0.1556657165149489